# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846910627369                   -0.70    4.8
  2   -7.852322602637       -2.27       -1.53    1.0
  3   -7.852646187373       -3.49       -2.52    3.0
  4   -7.852646677307       -6.31       -3.36    2.5
  5   -7.852646685813       -8.07       -4.74    1.2
  6   -7.852646686723       -9.04       -5.19    3.5
  7   -7.852646686730      -11.15       -6.09    1.5
  8   -7.852646686730      -12.54       -7.42    2.5
  9   -7.852646686730      -14.75       -7.58    2.2
 10   -7.852646686730   +    -Inf       -8.88    2.0
 11   -7.852646686730   +    -Inf       -9.67    2.2
 12   -7.852646686730   +  -15.05      -10.48    2.5
 13   -7.852646686730      -14.57      -11.01    1.5
 14   -7.852646686730   +  -15.05      -11.93    2.5
 15   -7.852646686730   +  -14.75      -12.42    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846747767487                   -0.70           4.2
  2   -7.852506231503       -2.24       -1.64   0.80    2.2
  3   -7.852632361543       -3.90       -2.71   0.80    1.0
  4   -7.852646452153       -4.85       -3.18   0.80    2.0
  5   -7.852646668513       -6.66       -3.94   0.80    1.2
  6   -7.852646686087       -7.76       -4.64   0.80    1.8
  7   -7.852646686705       -9.21       -5.51   0.80    1.8
  8   -7.852646686730      -10.62       -6.76   0.80    2.2
  9   -7.852646686730      -12.53       -7.10   0.80    3.0
 10   -7.852646686730      -14.75       -8.15   0.80    1.0
 11   -7.852646686730   +    -Inf       -9.15   0.80    2.2
 12   -7.852646686730   +    -Inf       -9.35   0.80    2.2
 13   -7.852646686730   +    -Inf      -10.33   0.80    1.0
 14   -7.852646686730   +  -14.75      -10.71   0.80    2.2
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.433102e+01     3.250408e+00
 * time: 0.5348060131072998
     1     9.178167e-01     1.708036e+00
 * time: 0.7858419418334961
     2    -1.756273e+00     2.059010e+00
 * time: 0.8213379383087158
     3    -3.906100e+00     1.639739e+00
 * time: 0.995089054107666
     4    -5.405724e+00     1.322613e+00
 * time: 1.045715093612671
     5    -6.938569e+00     9.263448e-01
 * time: 1.0973620414733887
     6    -7.131048e+00     1.175052e+00
 * time: 1.133862018585205
     7    -7.571928e+00     9.806713e-01
 * time: 1.1697051525115967
     8    -7.658917e+00     1.056881e+00
 * time: 1.206104040145874
     9    -7.729005e+00     9.398628e-01
 * time: 1.2437810897827148
    10    -7.745078e+00     1.015069e+00
 * time: 1.2799630165100098
    11    -7.767286e+00     8.806124e-01
 * time: 1.3159539699554443
    12    -7.791614e+00     1.822849e-01
 * time: 1.3671870231628418
    13    -7.808861e+00     1.603581e-01
 * time: 1.40392899513244

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846827472140                   -0.70    4.8
  2   -7.852316734576       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.54
  2   -7.852646686730      -10.69       -5.97
  3   -7.852646686730      -14.75      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.5942870111707984e-13
|ρ_newton - ρ_scfv| = 5.858834971953737e-13
|ρ_newton - ρ_dm|   = 1.9409433501383765e-10
